### la prédiction 'si le consommateur va cliquer le buy sur le site' en utilisant le jeu de données 'train'

In [11]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.decomposition import PCA

In [12]:
train = pd.read_csv("train.csv",sep=',')
print(train.shape)
print(train.columns)
print(train.dtypes)

(4107, 87)
Index(['Id', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31',
       'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41',
       'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51',
       'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61',
       'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71',
       'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81',
       'V82', 'V83', 'V84', 'V85', 'V86', 'Buy'],
      dtype='object')
Id     int64
V2     int64
V3     int64
V4     int64
V5     int64
       ...  
V83    int64
V84    int64
V85    int64
V86    int64
Buy    int64
Length: 87, dtype: object


In [13]:
train.head(5)

,Id,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V78,V79,V80,V81,V82,V83,V84,V85,V86,Buy
0,1,33,1,3,2,8,0,5,1,3,...,0,0,0,1,0,0,0,0,0,0
1,2,37,1,2,2,8,1,4,1,4,...,0,0,0,1,0,0,0,0,0,0
2,3,37,1,2,2,8,0,4,2,4,...,0,0,0,1,0,0,0,0,0,0
3,4,9,1,3,3,3,2,3,2,4,...,0,0,0,1,0,0,0,0,0,0
4,6,23,1,2,1,5,0,5,0,5,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#data1 = .to_numpy()
data = train.to_numpy()
print(data)
data_x = data[:,0:87]
data_y = data[:,-1]
print(data_x.shape)
print(data_y.shape)

[[   1   33    1 ...    0    0    0]
 [   2   37    1 ...    0    0    0]
 [   3   37    1 ...    0    0    0]
 ...
 [5819   35    1 ...    0    0    0]
 [5820   33    1 ...    0    0    1]
 [5822   33    1 ...    0    0    0]]
(4107, 87)
(4107,)


In [15]:
x_normalized = preprocessing.StandardScaler().fit_transform(data_x)
x_train,x_test,y_train, y_test = model_selection.train_test_split(x_normalized,data_y,test_size =0.3,random_state = 0)
pca = PCA(n_components = 80, whiten = True).fit(x_normalized)
data_x_pca = pca.transform(x_normalized)
x_train_pca,x_test_pca,y_train_pca, y_test_pca = model_selection.train_test_split(data_x_pca,data_y,test_size =0.3,
                                                                                  random_state = 0)

In [16]:
data_x_pca.shape

(4107, 80)

# Modele 0 SVM

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [18]:
x_test_pca.shape

(1233, 80)

In [19]:
x_train_pca.shape

(2874, 80)

In [20]:
#parametre a tester - jouer sur les noyaux et le 'cost parameter'
parametres = {"kernel": ['linear','poly'],"C":[0.1,0.5,1.0]}

#classifieru a utiliser
svmc = SVC()

#instanciation de la recherche
grille = GridSearchCV(estimator = svmc, param_grid = parametres,scoring = 'accuracy')

#lancer  l'exploration
resultats = grille.fit(x_train_pca,y_train_pca)

#meilleur parametrage :{}
print(resultats.best_params_)

#prediction avec le meilleur modele identifie
y_predc = resultats.predict(x_test_pca)

#performances du meilleur modele 
err_best = 1.0 - metrics.accuracy_score(y_test_pca,y_predc)
print(err_best)
print(metrics.accuracy_score(y_test_pca,y_predc))

{'C': 0.1, 'kernel': 'linear'}
0.0
1.0


In [21]:
TEST = pd.read_csv("test.csv",sep=',')
TEST = TEST.to_numpy()
TEST.shape

(1715, 86)

In [22]:
Norm_TestKaggle = preprocessing.StandardScaler().fit_transform(TEST)
scaled_test =pd.DataFrame(Norm_TestKaggle)
print(scaled_test.describe())
pca_fin = PCA(n_components = 80, whiten = True).fit(Norm_TestKaggle)
data_x_pca = pca_fin.transform(Norm_TestKaggle)

                 0             1             2             3             4   \
count  1.715000e+03  1.715000e+03  1.715000e+03  1.715000e+03  1.715000e+03   
mean   1.373699e-16 -2.052132e-17  3.755985e-16 -7.334593e-17 -4.425355e-16   
std    1.000292e+00  1.000292e+00  1.000292e+00  1.000292e+00  1.000292e+00   
min   -1.745704e+00 -1.825436e+00 -2.678980e-01 -2.059487e+00 -2.400840e+00   
25%   -8.682603e-01 -1.128658e+00 -2.678980e-01 -8.118642e-01 -1.198316e+00   
50%    7.988521e-03  4.197369e-01 -2.678980e-01  4.357587e-01  4.207079e-03   
75%    8.522706e-01  8.068356e-01 -2.678980e-01  4.357587e-01  4.207079e-03   
max    1.729416e+00  1.271354e+00  1.769923e+01  2.931004e+00  3.611777e+00   

                 5             6             7             8             9   \
count  1.715000e+03  1.715000e+03  1.715000e+03  1.715000e+03  1.715000e+03   
mean  -4.000687e-17  3.802595e-16 -2.191962e-16  5.101200e-17  1.633290e-16   
std    1.000292e+00  1.000292e+00  1.000292e+00  1.

In [23]:
y_predc_de_fichier_test = resultats.predict(data_x_pca)
#performances du meilleur modele 
print(y_predc_de_fichier_test)


[0 0 0 ... 0 0 0]


In [24]:
resultat_fin =pd.DataFrame(y_predc_de_fichier_test)
print(resultat_fin)
resultat_fin.describe()

      0
0     0
1     0
2     0
3     0
4     0
...  ..
1710  0
1711  0
1712  0
1713  0
1714  0

[1715 rows x 1 columns]


,0
count,1715.000000
mean,0.022741
std,0.149119
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [25]:
print(np.flatnonzero(y_predc_de_fichier_test))

[ 114  123  167  208  348  364  380  389  554  611  726  740  787  821
  825  847 1013 1058 1080 1128 1151 1169 1191 1219 1340 1347 1354 1355
 1435 1487 1613 1617 1640 1645 1659 1661 1666 1671 1701]


In [26]:
y_predc_de_fichier_test = pd.DataFrame(y_predc_de_fichier_test)
print(y_predc_de_fichier_test)

      0
0     0
1     0
2     0
3     0
4     0
...  ..
1710  0
1711  0
1712  0
1713  0
1714  0

[1715 rows x 1 columns]


In [27]:
with pd.ExcelWriter('Prediction_de_Ziyi.xlsx') as writer : 
    y_predc_de_fichier_test.to_excel(writer,sheet_name='Sheet1')